# KAN Architecture Sweep (Cos + Tumor)

This notebook runs KAN sweeps on the synthetic cosine and tumor datasets and reports
MSE, RMSE, MAE, MAPE, and parameter counts for each configuration.


In [2]:
import numpy as np
import pandas as pd
import torch
from pathlib import Path
import sys

repo_root = Path.cwd().parent 
sys.path.append(str(repo_root))

from mckan_pykan.config import MCKANPyKANConfig
from mckan_pykan.data import MCKANDataset, create_train_val_test_dataloaders
from mckan_pykan.training import training

from synt_data.generate_cos_data import generate_cos_dataset
from synt_data.generate_tumor_data import generate_tumor_dataset, prepare_static_history

/home/topsoe/frml/projects/TIMEVIEW/.venv/lib/python3.10/site-packages/lightning_fabric/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


In [6]:
def prepare_static_history_from_df(df, history_end=1.0):
    static_cols = [c for c in df.columns if c not in ('trajectory_id', 't', 'y')]
    trajectory_ids = sorted(df['trajectory_id'].unique())

    X_static = (
        df.drop_duplicates('trajectory_id')
        .sort_values('trajectory_id')[static_cols]
        .to_numpy()
    )

    Zs, ts, ys_list = [], [], []
    for tid in trajectory_ids:
        traj = df[df['trajectory_id'] == tid].sort_values('t')
        t_vals = traj['t'].to_numpy()
        y_vals = traj['y'].to_numpy()

        hist_mask = t_vals <= history_end
        pred_mask = t_vals > history_end

        Zs.append(y_vals[hist_mask].reshape(-1, 1))
        ts.append(t_vals[pred_mask].copy())
        ys_list.append(y_vals[pred_mask].copy())

    return X_static, Zs, ts, ys_list

def build_cos_dataset(n_samples=200, n_points=40, T=2.0, history_end=1.0, seed=42):
    df = generate_cos_dataset(n_samples=n_samples, n_points=n_points, seed=seed, T=T)
    return prepare_static_history_from_df(df, history_end=history_end), T

def build_tumor_dataset(n_samples=200, n_points=40, T=2.0, history_end=1.0, seed=0, equation='wilkerson'):
    df = generate_tumor_dataset(
        n_samples=n_samples,
        n_time_steps=n_points,
        time_horizon=T,
        seed=seed,
        equation=equation,
    )
    return prepare_static_history(df, history_end=history_end), T

In [7]:
def mse(a, b):
    return float(np.mean((a - b) ** 2))

def mae(a, b):
    return float(np.mean(np.abs(a - b)))

def mape(a, b, eps=1e-8):
    return float(np.mean(np.abs((a - b) / (np.abs(b) + eps))))

def eval_loader_metrics(model, loader):
    mse_vals, mae_vals, mape_vals = [], [], []
    model.eval()
    with torch.no_grad():
        for batch in loader:
            Xb, Zb, Phis, yb = batch
            preds = model(Xb, Zb, Phis)
            for pred, y in zip(preds, yb):
                pred_np = pred.cpu().numpy()
                y_np = y.cpu().numpy()
                mse_vals.append(mse(pred_np, y_np))
                mae_vals.append(mae(pred_np, y_np))
                mape_vals.append(mape(pred_np, y_np))
    mse_mean = float(np.mean(mse_vals))
    mae_mean = float(np.mean(mae_vals))
    mape_mean = float(np.mean(mape_vals))
    rmse_mean = float(np.sqrt(mse_mean))
    return mse_mean, rmse_mean, mae_mean, mape_mean

def count_params(model):
    return int(sum(p.numel() for p in model.parameters()))

def run_kan_sweep(dataset_name, X_static, Zs, ts, ys_list, T, widths_list, grids, n_basis_list, max_epochs=200, seed=42):
    results = []
    history_len = Zs[0].shape[0]
    dynamic_feature_dim = Zs[0].shape[1]
    n_dynamic_features = history_len * dynamic_feature_dim

    for widths in widths_list:
        for grid in grids:
            for n_basis in n_basis_list:
                cfg = MCKANPyKANConfig(
                    n_static_features=X_static.shape[1],
                    n_dynamic_features=n_dynamic_features,
                    n_basis=n_basis,
                    T=T,
                    dynamic_mode='history',
                    num_epochs=max_epochs,
                    kan={
                        'widths': list(widths),
                        'grid': grid,
                        'degree': 3,
                        'kwargs': {},
                    },
                )

                dataset = MCKANDataset(cfg, (X_static, Zs, ts, ys_list))
                litmodel = training(cfg.seed, cfg, dataset)
                model = litmodel.model

                _, _, test_loader = create_train_val_test_dataloaders(cfg, dataset)
                mse_val, rmse_val, mae_val, mape_val = eval_loader_metrics(model, test_loader)
                params = count_params(model)

                results.append({
                    'Dataset': dataset_name,
                    'Model': f'KAN widths={widths} grid={grid} n_basis={n_basis}',
                    'MSE': mse_val,
                    'RMSE': rmse_val,
                    'MAE': mae_val,
                    'MAPE': mape_val,
                    'Parameters': params,
                })
                print(dataset_name, widths, grid, n_basis, 'MSE', mse_val, 'RMSE', rmse_val, 'MAE', mae_val, 'MAPE', mape_val, 'params', params)
    return pd.DataFrame(results)


In [8]:
widths_list = [(8, 8), (16, 16), (32, 32)]
grids = [2, 6, 8]
n_basis_list = [6, 8, 10]

history_end = 1.0
T = 2.0
n_samples = 200
n_points = 40

In [9]:
(X_static_cos, Zs_cos, ts_cos, ys_cos), T_cos = build_cos_dataset(
    n_samples=n_samples,
    n_points=n_points,
    T=T,
    history_end=history_end,
    seed=42,
)

cos_results = run_kan_sweep(
    dataset_name='cos',
    X_static=X_static_cos,
    Zs=Zs_cos,
    ts=ts_cos,
    ys_list=ys_cos,
    T=T_cos,
    widths_list=widths_list,
    grids=grids,
    n_basis_list=n_basis_list,
    max_epochs=200,
)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


/home/topsoe/frml/projects/TIMEVIEW/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/topsoe/frml/projects/TIMEVIEW/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=200` reached.
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: F

cos (8, 8) 2 6 MSE 0.1285812560672639 RMSE 0.3585822863266727 MAE 0.21390145039185882 MAPE 0.5508002948015929 params 5750
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


`Trainer.fit` stopped: `max_epochs=200` reached.
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

cos (32, 32) 8 6 MSE 0.10492152835038268 RMSE 0.32391592790473067 MAE 0.16450034263543784 MAPE 0.3520340888993815 params 61772
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=200` reached.
Global seed set to 42


cos (32, 32) 8 8 MSE 0.09196868151489071 RMSE 0.3032633863737769 MAE 0.13480394572252408 MAPE 0.2687825637869537 params 64220
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=200` reached.


cos (32, 32) 8 10 MSE 0.0762452668470928 RMSE 0.27612545490608575 MAE 0.11147485293913632 MAPE 0.22796070624608547 params 66668


In [10]:
# Run sweep on tumor dataset
(X_static_tumor, Zs_tumor, ts_tumor, ys_tumor), T_tumor = build_tumor_dataset(
    n_samples=n_samples,
    n_points=n_points,
    T=T,
    history_end=history_end,
    seed=0,
    equation='wilkerson',
)

tumor_results = run_kan_sweep(
    dataset_name='tumor',
    X_static=X_static_tumor,
    Zs=Zs_tumor,
    ts=ts_tumor,
    ys_list=ys_tumor,
    T=T_tumor,
    widths_list=widths_list,
    grids=grids,
    n_basis_list=n_basis_list,
    max_epochs=200,
)


Global seed set to 42


checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/topsoe/frml/projects/TIMEVIEW/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/topsoe/frml/projects/TIMEVIEW/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
`Trainer.fit` s

tumor (8, 8) 2 6 MSE 10.149611405283213 RMSE 3.185845477307902 MAE 1.8298542767763137 MAPE 70.85127738714218 params 6089
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


`Trainer.fit` stopped: `max_epochs=200` reached.
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

tumor (32, 32) 8 6 MSE 0.7881116646341979 RMSE 0.8877565345488581 MAE 0.4945080939680338 MAPE 13.399771755933761 params 63641
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=200` reached.
Global seed set to 42


tumor (32, 32) 8 8 MSE 0.9053155953530222 RMSE 0.9514807382984809 MAE 0.48927762024104593 MAPE 17.48955816105008 params 66089
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=200` reached.


tumor (32, 32) 8 10 MSE 0.7059069961309433 RMSE 0.8401827159201404 MAE 0.45225470662117007 MAPE 9.971461250726133 params 68537


In [12]:
def format_table(df):
    out = df.copy()
    out['MSE (×10^-3)'] = out['MSE'] * 1e3
    out['RMSE (×10^-2)'] = out['RMSE'] * 1e2
    out['MAE (×10^-2)'] = out['MAE'] * 1e2
    out['MAPE'] = out['MAPE']
    cols = ['Model', 'MSE (×10^-3)', 'RMSE (×10^-2)', 'MAE (×10^-2)', 'MAPE', 'Parameters']
    return out[cols]

print('Cos dataset results')
display(format_table(cos_results))

print('Tumor dataset results')
display(format_table(tumor_results))


Cos dataset results


,Model,MSE (×10^-3),RMSE (×10^-2),MAE (×10^-2),MAPE,Parameters
0,"KAN widths=(8, 8) grid=2 n_basis=6",128.581256,35.858229,21.390145,0.550800,5750
1,"KAN widths=(8, 8) grid=2 n_basis=8",97.851657,31.281249,15.497017,0.325383,6182
2,"KAN widths=(8, 8) grid=2 n_basis=10",99.665357,31.569821,15.640964,0.365183,6614
3,"KAN widths=(8, 8) grid=6 n_basis=6",108.554284,32.947577,17.692730,0.368327,7530
4,"KAN widths=(8, 8) grid=6 n_basis=8",100.513356,31.703841,15.397640,0.338231,8090
5,"KAN widths=(8, 8) grid=6 n_basis=10",89.756209,29.959341,13.086593,0.274659,8650
6,"KAN widths=(8, 8) grid=8 n_basis=6",121.855970,34.907874,18.348757,0.406895,8420
7,"KAN widths=(8, 8) grid=8 n_basis=8",99.456617,31.536743,14.401950,0.311071,9044
8,"KAN widths=(8, 8) grid=8 n_basis=10",101.120891,31.799511,13.708538,0.312916,9668
9,"KAN widths=(16, 16) grid=2 n_basis=6",107.743117,32.824247,17.508362,0.386043,14590


Tumor dataset results


,Model,MSE (×10^-3),RMSE (×10^-2),MAE (×10^-2),MAPE,Parameters
0,"KAN widths=(8, 8) grid=2 n_basis=6",10149.611405,318.584548,182.985428,70.851277,6089
1,"KAN widths=(8, 8) grid=2 n_basis=8",12734.971082,356.860912,188.521348,71.242317,6521
2,"KAN widths=(8, 8) grid=2 n_basis=10",14572.464170,381.738971,195.778900,80.366093,6953
3,"KAN widths=(8, 8) grid=6 n_basis=6",1043.322008,102.143135,68.289728,17.388018,7977
4,"KAN widths=(8, 8) grid=6 n_basis=8",9741.121498,312.107698,165.997148,50.448587,8537
5,"KAN widths=(8, 8) grid=6 n_basis=10",46275.817907,680.263316,354.784741,137.140283,9097
6,"KAN widths=(8, 8) grid=8 n_basis=6",3691.895889,192.143069,95.459879,38.858786,8921
7,"KAN widths=(8, 8) grid=8 n_basis=8",1914.843514,138.377871,68.500230,23.133673,9545
8,"KAN widths=(8, 8) grid=8 n_basis=10",6998.366845,264.544266,121.750822,45.668539,10169
9,"KAN widths=(16, 16) grid=2 n_basis=6",2090.197238,144.575144,86.729572,20.903178,15241


In [14]:
format_table(cos_results).to_csv('cos_results.csv', index=False)
format_table(tumor_results).to_csv('tumor_results.csv', index=False)

In [18]:
cos_results.to_csv('cos_results_orig.csv', index=False)
tumor_results.to_csv('tumor_results_orig.csv', index=False)

In [19]:
cos_results

,Dataset,Model,MSE,RMSE,MAE,MAPE,Parameters
0,cos,"KAN widths=(8, 8) grid=2 n_basis=6",0.128581,0.358582,0.213901,0.550800,5750
1,cos,"KAN widths=(8, 8) grid=2 n_basis=8",0.097852,0.312812,0.154970,0.325383,6182
2,cos,"KAN widths=(8, 8) grid=2 n_basis=10",0.099665,0.315698,0.156410,0.365183,6614
3,cos,"KAN widths=(8, 8) grid=6 n_basis=6",0.108554,0.329476,0.176927,0.368327,7530
4,cos,"KAN widths=(8, 8) grid=6 n_basis=8",0.100513,0.317038,0.153976,0.338231,8090
5,cos,"KAN widths=(8, 8) grid=6 n_basis=10",0.089756,0.299593,0.130866,0.274659,8650
6,cos,"KAN widths=(8, 8) grid=8 n_basis=6",0.121856,0.349079,0.183488,0.406895,8420
7,cos,"KAN widths=(8, 8) grid=8 n_basis=8",0.099457,0.315367,0.144019,0.311071,9044
8,cos,"KAN widths=(8, 8) grid=8 n_basis=10",0.101121,0.317995,0.137085,0.312916,9668
9,cos,"KAN widths=(16, 16) grid=2 n_basis=6",0.107743,0.328242,0.175084,0.386043,14590


In [20]:
tumor_results

,Dataset,Model,MSE,RMSE,MAE,MAPE,Parameters
0,tumor,"KAN widths=(8, 8) grid=2 n_basis=6",10.149611,3.185845,1.829854,70.851277,6089
1,tumor,"KAN widths=(8, 8) grid=2 n_basis=8",12.734971,3.568609,1.885213,71.242317,6521
2,tumor,"KAN widths=(8, 8) grid=2 n_basis=10",14.572464,3.817390,1.957789,80.366093,6953
3,tumor,"KAN widths=(8, 8) grid=6 n_basis=6",1.043322,1.021431,0.682897,17.388018,7977
4,tumor,"KAN widths=(8, 8) grid=6 n_basis=8",9.741121,3.121077,1.659971,50.448587,8537
5,tumor,"KAN widths=(8, 8) grid=6 n_basis=10",46.275818,6.802633,3.547847,137.140283,9097
6,tumor,"KAN widths=(8, 8) grid=8 n_basis=6",3.691896,1.921431,0.954599,38.858786,8921
7,tumor,"KAN widths=(8, 8) grid=8 n_basis=8",1.914844,1.383779,0.685002,23.133673,9545
8,tumor,"KAN widths=(8, 8) grid=8 n_basis=10",6.998367,2.645443,1.217508,45.668539,10169
9,tumor,"KAN widths=(16, 16) grid=2 n_basis=6",2.090197,1.445751,0.867296,20.903178,15241
